In [118]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os

pd.options.display.max_columns = None

import pickle

In [119]:
# t_seps = [10, 12, 14, 16, 18, 20, 22, 24]
t_seps = [12, 14, 16, 18]

# trajs = range(2100, 2150, 10)
trajs = sorted([int(f[:4]) for f in os.listdir('./Kaon_results_control_R/')]) 
trajs = [traj for traj in trajs if 1870<=traj<=2180]

print(trajs)
print(f'Number of configurations {len(trajs)}')


def mean_err(arr):  # shape: (trajs, ...)
    mean = arr.mean(axis=0)
    err = arr.std(axis=0) / np.sqrt(arr.shape[0])  
    return mean, err  

[1870, 1890, 1900, 1910, 1930, 1940, 1950, 1960, 1970, 1990, 2000, 2010, 2020, 2030, 2040, 2050, 2060, 2070, 2080, 2090, 2100, 2110, 2120, 2130, 2140, 2150, 2160, 2170, 2180]
Number of configurations 29


In [120]:
# traj = 2000
# data = pickle.load(open(f'./Kaon_results/{traj}.pkl', 'rb'))
# data.keys()

In [121]:
# With disconnected diagrams
coefficients = {
    'cs_sBar_d_T1D1': -1/np.sqrt(6), 'cs_sBar_d_T1D2': 2/np.sqrt(6), 
    
    'cs_sBar_d_T2D1': 2/np.sqrt(6), 'cs_sBar_d_T2D2': -2/np.sqrt(6),   # disconnected
    
    'cs_Hw_T1D1Q1': 1/np.sqrt(6), 'cs_Hw_T1D1Q2': -1/np.sqrt(6), 
    'cs_Hw_T2D1Q1': 1/np.sqrt(6), 'cs_Hw_T2D1Q2': -1/np.sqrt(6), 
    'cs_Hw_T2D2Q1': -2/np.sqrt(6), 'cs_Hw_T2D2Q2': 2/np.sqrt(6), 
    
    'cs_Hw_T3D1Q1': -2/np.sqrt(6), 'cs_Hw_T3D1Q2': 2/np.sqrt(6),    # disconnected
    'cs_Hw_T3D2Q1': 2/np.sqrt(6), 'cs_Hw_T3D2Q2': -2/np.sqrt(6)   # disconnected
}


diagrams = list(coefficients.keys())

Q1_diagrams = [d for d in diagrams if d.endswith('Q1')] 
Q2_diagrams = [d for d in diagrams if d.endswith('Q2')] 
sBar_d_diagrams = [d for d in diagrams if d.startswith('cs_sBar_d')]

print(f'Q1_diagrams: {Q1_diagrams}')
print(f'Q2_diagrams: {Q2_diagrams}')
print(f'sBar_d_diagrams: {sBar_d_diagrams}')

Q1_diagrams: ['cs_Hw_T1D1Q1', 'cs_Hw_T2D1Q1', 'cs_Hw_T2D2Q1', 'cs_Hw_T3D1Q1', 'cs_Hw_T3D2Q1']
Q2_diagrams: ['cs_Hw_T1D1Q2', 'cs_Hw_T2D1Q2', 'cs_Hw_T2D2Q2', 'cs_Hw_T3D1Q2', 'cs_Hw_T3D2Q2']
sBar_d_diagrams: ['cs_sBar_d_T1D1', 'cs_sBar_d_T1D2', 'cs_sBar_d_T2D1', 'cs_sBar_d_T2D2']


In [122]:
def calc_cs(traj):
    data = pickle.load(open(f'./Kaon_results_control_R/{traj}.pkl', 'rb'))
    
    eta_Q1_K = sum([data[d] * coefficients[d] for d in Q1_diagrams])
    eta_Q2_K = sum([data[d] * coefficients[d] for d in Q2_diagrams])
    eta_sBar_d_K = sum([data[d] * coefficients[d] for d in sBar_d_diagrams])
    
    eta_Q1_K = eta_Q1_K.real # ??? should I take real part; seems like imag part is small
    eta_Q2_K = eta_Q2_K.real 
    eta_sBar_d_K = eta_sBar_d_K.real
    
    return eta_Q1_K, eta_Q2_K, eta_sBar_d_K # shape (tsep, tx, R)

#     c_s_Q1 = -  eta_Q1_K / eta_sBar_d_K
#     c_s_Q2 = -  eta_Q2_K / eta_sBar_d_K
    
#     return c_s_Q1, c_s_Q2   # shape (tsep, tx)

In [133]:
eta_Q1_K, eta_Q2_K, eta_sBar_d_K = [], [], []

for traj in trajs:
    eta_Q1_K_one_traj, eta_Q2_K_one_traj, eta_sBar_d_K_one_traj = calc_cs(traj)   # shape (tsep, tx, R)
#     c_s_Q1_one_traj, c_s_Q2_one_traj = calc_cs(traj)   # shape (tsep, tx)
    eta_Q1_K.append(eta_Q1_K_one_traj)
    eta_Q2_K.append(eta_Q2_K_one_traj)
    eta_sBar_d_K.append(eta_sBar_d_K_one_traj)
eta_Q1_K = np.array(eta_Q1_K)  # (trajs, tsep, tx, R)
eta_Q2_K = np.array(eta_Q2_K)
eta_sBar_d_K = np.array(eta_sBar_d_K)

eta_Q1_K = eta_Q1_K.sum(axis=-1) # sum over R # get (tsep, tx)
eta_Q2_K = eta_Q2_K.sum(axis=-1) # sum over R # get (tsep, tx)
eta_sBar_d_K = eta_sBar_d_K.sum(axis=-1) # sum over R # get (tsep, tx)

eta_Q1_K_avg, eta_Q1_K_err = mean_err(eta_Q1_K)
eta_Q2_K_avg, eta_Q2_K_err = mean_err(eta_Q2_K)
eta_sBar_d_K_avg, eta_sBar_d_K_err = mean_err(eta_sBar_d_K)

c_s_Q1 = -  eta_Q1_K_avg / eta_sBar_d_K_avg
c_s_Q2 = -  eta_Q2_K_avg / eta_sBar_d_K_avg

In [134]:
eta_Q1_K_avg.shape

(4, 19)

In [136]:
def plot_by_tsep(avg, err, title='xxxxx'):
    fig = go.Figure()
    for i, t_sep in enumerate(t_seps[:5]):
        fig.add_trace(
            go.Scatter(x=list(range(t_sep)), y=avg[i][:t_sep], name=f'tsep={t_sep}',
                      error_y=dict(type='data', array=err[i][:t_sep]))
        )
    fig.update_layout(title_text=title, width=600, height=400)
#     fig.show()
    return fig

plot_by_tsep(eta_Q1_K_avg, eta_Q1_K_err).show()
plot_by_tsep(eta_sBar_d_K_avg, eta_sBar_d_K_err).show()

In [125]:
# def plot_by_tsep(cs, title='xxxxx'):
#     fig = go.Figure()
# #     for i, t_sep in enumerate(t_seps[3:]):
#     for i, t_sep in enumerate(t_seps[:5]):
#         fig.add_trace(
#             go.Scatter(x=list(range(t_sep)), y=cs[i][:t_sep], name=f'tsep={t_sep}')
#         )
#     fig.update_layout(title_text=title, width=600, height=400)
# #     fig.show()
#     return fig

# plot_by_tsep(c_s_Q1).show()
# plot_by_tsep(c_s_Q2).show()

In [139]:
eta_Q1_K, eta_Q2_K, eta_sBar_d_K = [], [], []

for traj in trajs:
    eta_Q1_K_one_traj, eta_Q2_K_one_traj, eta_sBar_d_K_one_traj = calc_cs(traj)   # shape (tsep, tx, R)
#     c_s_Q1_one_traj, c_s_Q2_one_traj = calc_cs(traj)   # shape (tsep, tx)
    eta_Q1_K.append(eta_Q1_K_one_traj)
    eta_Q2_K.append(eta_Q2_K_one_traj)
    eta_sBar_d_K.append(eta_sBar_d_K_one_traj)
eta_Q1_K = np.array(eta_Q1_K)  # (trajs, tsep, tx, R)
eta_Q2_K = np.array(eta_Q2_K)
eta_sBar_d_K = np.array(eta_sBar_d_K)

eta_Q1_K = eta_Q1_K.cumsum(axis=-1) # get cumulative sum of R
eta_Q2_K = eta_Q2_K.cumsum(axis=-1) # get cumulative sum of R
eta_sBar_d_K = eta_sBar_d_K.cumsum(axis=-1) # get cumulative sum of R

eta_Q1_K_avg, eta_Q1_K_err = mean_err(eta_Q1_K)  # [tsep, t_x, R]
eta_Q2_K_avg, eta_Q2_K_err = mean_err(eta_Q2_K)
eta_sBar_d_K_avg, eta_sBar_d_K_err = mean_err(eta_sBar_d_K)



def plot_by_tsep(avg, err, title='xxxxx'):
    fig = go.Figure()
    for i, t_sep in enumerate(t_seps[:5]):
        fig.add_trace(
            go.Scatter(x=list(range(len(avg[i]))), y=avg[i], name=f'tsep={t_sep}',
                      error_y=dict(type='data', array=err[i]))
        )
    fig.update_layout(title_text='<eta Q1 K> vs max R', width=600, height=400, xaxis_title='max R',
                     yaxis_title='<eta Q1 K>')
#     fig.show()
    return fig

plot_by_tsep(eta_Q1_K_avg[:, 8, :], eta_Q1_K_err[:, 8, :])

In [127]:
assert False

AssertionError: 

In [ ]:
# eta_Q1_K = eta_Q1_K[:, :, 8, :] # get tx=8 # get (trajs, tsep, R)
# eta_Q2_K = eta_Q2_K[:, :, 8, :] # get tx=8 # get (trajs, tsep, R)
# eta_sBar_d_K = eta_sBar_d_K[:, :, 8, :] # get tx=8 # get (trajs, tsep, R)

# eta_Q1_K_avg, eta_Q1_K_err = mean_err(eta_Q1_K)
# eta_Q2_K_avg, eta_Q2_K_err = mean_err(eta_Q2_K)
# eta_sBar_d_K_avg, eta_sBar_d_K_err = mean_err(eta_sBar_d_K)

# c_s_Q1_avg = -  eta_Q1_K_avg / eta_sBar_d_K_avg
# c_s_Q2_avg = -  eta_Q2_K_avg / eta_sBar_d_K_avg
# c_s_Q1_err = np.sqrt(abs(c_s_Q1_avg) * ((eta_Q1_K_err / eta_Q1_K_avg)**2 + (eta_sBar_d_K_err / eta_sBar_d_K_avg)**2) )
# c_s_Q2_err = np.sqrt(abs(c_s_Q2_avg) * ((eta_Q2_K_err / eta_Q2_K_avg)**2 + (eta_sBar_d_K_err / eta_sBar_d_K_avg)**2) )

In [ ]:
for i, t_sep in enumerate(t_seps):
    c_s_Q1_avg[i, t_sep-2:], c_s_Q1_err[i, t_sep-2:] = np.nan, np.nan
    c_s_Q2_avg[i, t_sep-2:], c_s_Q2_err[i, t_sep-2:] = np.nan, np.nan    

weights = 1 / (c_s_Q1_err**2)
weights =  weights / np.nansum(weights, axis=0)

c_s_Q1_avg_avg_tsep = np.nansum(c_s_Q1_avg * weights, axis=0) # average over tsep
c_s_Q1_err_avg_tsep = np.nansum((c_s_Q1_err * weights)**2, axis=0)
c_s_Q2_avg_avg_tsep = np.nansum(c_s_Q2_avg * weights, axis=0) # average over tsep
c_s_Q2_err_avg_tsep = np.nansum((c_s_Q2_err * weights)**2, axis=0)

In [ ]:
# TBD: cumulative sum over R axis

In [ ]:
c_s_Q1_avg.shape

In [ ]:
def plot_final_cs(cs_avg, cs_err, title='xxxxx'):
    fig = go.Figure()
#     for i, t_sep in enumerate(t_seps[3:]):
    fig.add_trace(
        go.Scatter(x=list(range(len(cs_avg))), y=cs_avg, error_y_array=cs_err)
    )
    fig.update_layout(title_text=title, width=600, height=400,
                     yaxis_title='c_s', xaxis_title='max R')
    fig.update_layout()
    return fig

plot_final_cs(c_s_Q1_avg_avg_tsep, c_s_Q1_err_avg_tsep, title='c_s Q1').show()